[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ginamanou/avalanche-susceptibility/blob/main/7_Feature_selection_model_development_testing_and_comparisons.ipynb)

## Introduction

This notebook demonstrates the process of data-driven model development, attempting to solve a classification problem.

It includes processes that need to precede the model development, like feature selection and splitting into training, validation and test sets.

The developed models are:

1) Decesion Tree
2) Random Forest
3) Extra Trees
4) Adaboost
5) Gradient Boosting
6) XGBoost
7) Support Vector Machine
8) Logistic Regression
9) Feedforward Neural Network

Each model is trained, optimized, validated and then tested to check its performance. Once all models are deployed using the whole available dataset, the performances are compared using **heatmaps** and **ROC curves**.

## Import the necessary packages

In [ ]:
# General packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline

In [ ]:
# Preprocessing / Feature engineering
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

In [ ]:
# Model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold

In [ ]:
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

In [ ]:
# Evaluation metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score, precision_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.metrics import precision_score, recall_score

In [ ]:
# Feature selection
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
# Special packages to plot trees
import graphviz 
import pydotplus
import pylab
import colour
from dtreeviz import trees
from dtreeviz.trees import *

In [ ]:
# Models

# Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.tree import _tree

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Extra Trees
from sklearn.ensemble import ExtraTreesClassifier

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier

# Gradient Boost
from sklearn.ensemble import GradientBoostingClassifier

# XGBoost
import xgboost as xgb

# SVM
from sklearn.svm import SVC

# Neural Network
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Logistic Regression
from sklearn.linear_model import LogisticRegression

## Import the data

In [ ]:
# Here I can just load the respective preprocessed dataset/file from the folder, 
# since I have done the preprocessing already in another script

df = pd.read_csv("D:/Allaus/Code/train/2_both_based_on_ptz/DataBase_IDW_preprocessed.csv")
df

In [ ]:
df.columns

In [ ]:
len(df.columns)

Some explorative plots

In [ ]:
for col in df.iloc[:,:]:
    plt.hist(df[df['Avalanche'] == 1][col], color='red', label='Susceptible', alpha=0.7, density=True) # density normalizes these distributions
    plt.hist(df[df['Avalanche'] == 0][col], color='blue', label='Not susceptible', alpha=0.7, density=True)
    plt.title(col)
    plt.ylabel('Probability')
    plt.xlabel(col)
    plt.legend()
    plt.show();
    

In [ ]:
# # Alternative plotting
# sns.pairplot(df_encoded, hue='Allaus', diag_kind='hist')
# #plt.savefig('D:/Allaus/Data_analysis/ML/pairplot_ptz.png', dpi=300)
# plt.show();

## Split between dependent and independent / target variables

In [ ]:
# Including the Area as a predictor is wrong, because I manipulated it when I was creating the non-allaus data, 
# it didn't come from the raw data

X = df.drop(['Avalanche'], axis=1).copy()   
X.head()

In [ ]:
X.tail()

In [ ]:
y = df['Avalanche'].copy()
y.head()

In [ ]:
X.shape, y.shape

In [ ]:
X.dtypes

In [ ]:
y.unique()

In [ ]:
X = X.drop(columns=['OrientationToNorth', 'PotentialSnowTransport', 'Aspect_East', 'Aspect_North',
       'Aspect_North-East', 'Aspect_North-West',
       'Aspect_South-East', 'Aspect_South-West', 'Aspect_West', 'Wind_East',
       'Wind_North', 'Wind_North-East', 'Wind_North-West',
       'Wind_South-East', 'Wind_South-West', 'Wind_West', 'Aspect_South'])
X

## Shuffle the data

Shuffling may be bad for reproducibility, but it is good for reliability!!!

In [ ]:
# df = df.sample(frac = 1)  # in this case, I do it together with the train, valid, test split as seen below

## Train, validation and test set split

In [ ]:
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
valid.head()

In [ ]:
test.head()

In [ ]:
X_train = train.drop(['Avalanche'], axis=1).copy()
y_train = train['Avalanche'].copy()

In [ ]:
X_valid = valid.drop(['Avalanche'], axis=1).copy()
y_valid = valid['Avalanche'].copy()

In [ ]:
X_test = test.drop(['Avalanche'], axis=1).copy()
y_test = test['Avalanche'].copy()

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_valid.shape, y_valid.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
y_train.unique(), y_valid.unique(), y_test.unique()

In [ ]:
# Make sure that the classes are more or less balanced

plt.hist(y_train)
plt.ylabel("Count of output labels")
plt.show()

plt.hist(y_valid)
plt.ylabel("Count of output labels")
plt.show()

plt.hist(y_test)
plt.ylabel("Count of output labels")
plt.show()

## Feature selection

In [ ]:
X_train.info()

In [ ]:
X_train.describe().T

**Variance Threshold**

In this case it is not an issue.

I don't have features with constant values.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
var_thres = VarianceThreshold(threshold=0)
var_thres.fit(X_train)

In [ ]:
var_thres.get_support()    # If all are true, then none of the variables is constant

**Chi-2 Test**

!!! Only for categorical (non-negative) variables !!!

It returns two values: F-score & p-value.

The higher the F-score, the more important the feature is.

In [ ]:
X_train.columns

In [ ]:
len(X_train.columns)

In [ ]:
# Select only the categorical variables

X_chi2 = X_train[['OrientationToNorth', 'Forest', 'Rocks', 'Screes', 
                  'PotentialSnowTransport', 'Aspect_East', 'Aspect_North',
                   'Aspect_North-East', 'Aspect_North-West', 'Aspect_South',
                   'Aspect_South-East', 'Aspect_South-West', 'Aspect_West', 'Wind_East',
                   'Wind_North', 'Wind_North-East', 'Wind_North-West', 'Wind_South',
                   'Wind_South-East', 'Wind_South-West', 'Wind_West']]
len(X_chi2.columns)

In [ ]:
y_chi2 = y_train.copy()
y_chi2

In [ ]:
y_chi2.unique()

In [ ]:
from sklearn.feature_selection import chi2
f_p_values = chi2(X_chi2, y_chi2)
f_p_values

In [ ]:
p_values = pd.Series(f_p_values[1])
p_values.index = X_chi2.columns
p_values

In [ ]:
p_values.sort_values(ascending=True)

**Get rid of categorical values that are not important**

Based on the Chi2 test.

I remove every variable with p-value < 0.05.

Alternative: use the SelectKBest object or the nlargest() function.

In [ ]:
# # I can use this in order to select the features that I want after every test
# from sklearn.feature_selection import SelectKBest

# sel_feat = SelectKBest(chi2, k = 8)
# sel_feat.fit(X_chi2, y_chi2)
# X_chi2.columns[sel_feat.get_support()]

# # nlargest() is also an option if I have the scores or the p-values
# p_values.nsmallest(8)

**At first, mostly based on the histograms, remove the least important categorical features**

In [ ]:
X_train.columns

In [ ]:
X_train = X_train.drop(columns=['OrientationToNorth', 'PotentialSnowTransport', 'Aspect_East', 'Aspect_North',
       'Aspect_North-East', 'Aspect_North-West',
       'Aspect_South-East', 'Aspect_South-West', 'Aspect_West', 'Wind_East',
       'Wind_North', 'Wind_North-East', 'Wind_North-West',
       'Wind_South-East', 'Wind_South-West', 'Wind_West'])
X_train


In [ ]:
X_train.columns

In [ ]:
y_train.unique()

In [ ]:
y_train.shape

**Do the same for the validation and test set**

In [ ]:
X_valid

In [ ]:
X_valid = X_valid.drop(columns=['OrientationToNorth', 'PotentialSnowTransport', 'Aspect_East', 'Aspect_North',
       'Aspect_North-East', 'Aspect_North-West',
       'Aspect_South-East', 'Aspect_South-West', 'Aspect_West', 'Wind_East',
       'Wind_North', 'Wind_North-East', 'Wind_North-West',
       'Wind_South-East', 'Wind_South-West', 'Wind_West'])
X_valid


In [ ]:
y_valid.unique()

In [ ]:
y_valid.shape

In [ ]:
X_test = X_test.drop(columns=['OrientationToNorth', 'PotentialSnowTransport', 'Aspect_East', 'Aspect_North',
       'Aspect_North-East', 'Aspect_North-West',
       'Aspect_South-East', 'Aspect_South-West', 'Aspect_West', 'Wind_East',
       'Wind_North', 'Wind_North-East', 'Wind_North-West',
       'Wind_South-East', 'Wind_South-West', 'Wind_West'])
X_test


In [ ]:
y_test.shape

**Pearson's Correlation**

In [ ]:
# Plot color scaled correlation matrix

corr=X_train.corr()
corr.style.background_gradient(cmap='coolwarm')       

In [ ]:
X_train.ExtremeRain.corr(X_train.MonthlyRain)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 12))
sns.heatmap(X_train.corr())
plt.title("Feature correlation heatmap", y=1.04)
plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/correlation - independent variables.png', dpi=300)
plt.show();

In [ ]:
df.SnowTotal.corr(df.Tmax)

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i, j] > threshold:   # I want to keep the negative correlations
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(X_train, 0.85)
len(set(corr_features))

In [ ]:
corr_features

**Mutual Information**

Mutual information is sometimes used as a synonym for information gain. 
Technically, they calculate the same quantity if applied to the same data.

In [ ]:
from sklearn.feature_selection import mutual_info_classif
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending = False)

In [ ]:
X_train.ExtremeRain.corr(X_train.Rain)

In [ ]:
X_train.WindSpeedMean.corr(X_train.WindSpeedMax)

In [ ]:
X_train.Slope.corr(X_train.CurvatureProf)

In [ ]:
X_train.Tmean.corr(X_train.Trange)

In [ ]:
mutual_info.sort_values(ascending=False).plot.bar(figsize=(20,8))
plt.title('Mutual Information')
plt.xlabel("Features")
plt.ylabel("Scores")
plt.show();

**Extra Trees - feature importance**

In [ ]:
clf_et = ExtraTreesClassifier(criterion='entropy', random_state=5)
clf_et.fit(X_train, y_train)

In [ ]:
# Feature importances
clf_et.feature_importances_

In [ ]:
features = X_train.columns
importances=clf_et.feature_importances_
indices=np.argsort(importances)

plt.figure(figsize=(10, 12))
plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color='g', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel("Relative importance")
plt.show()

**Remove more irrelevant features.**

**Do it for train, valid and test sets.**

In [ ]:
X_train = X_train.drop(columns=['Aspect_South'])
X_train

In [ ]:
X_valid = X_valid.drop(columns=['Aspect_South'])
X_valid


In [ ]:
X_test = X_test.drop(columns=['Aspect_South'])
X_test


In [ ]:
y_test.shape, y_valid.shape, y_train.shape

In [ ]:
y_test.unique(), y_valid.unique(), y_train.unique()

In [ ]:
X_train.columns

**Save the train, valid, test sets for reproducibility of results during the stage of finalization and writing.**

In [ ]:
X_train.to_csv("D:/Allaus/Code/train/2_both_based_on_ptz/X_train.csv")

In [ ]:
X_valid.to_csv("D:/Allaus/Code/train/2_both_based_on_ptz/X_valid.csv")

In [ ]:
X_test.to_csv("D:/Allaus/Code/train/2_both_based_on_ptz/X_test.csv")

In [ ]:
y_train.to_csv("D:/Allaus/Code/train/2_both_based_on_ptz/y_train.csv")

In [ ]:
y_valid.to_csv("D:/Allaus/Code/train/2_both_based_on_ptz/y_valid.csv")

In [ ]:
y_test.to_csv("D:/Allaus/Code/train/2_both_based_on_ptz/y_test.csv")

## Data import

Read directly the finalized datasets as of after the feature selection performed above

In [ ]:
X_train = pd.read_csv("D:/Allaus/Code/train/2_both_based_on_ptz/X_train.csv", index_col='Unnamed: 0')
X_train

In [ ]:
X_valid = pd.read_csv("D:/Allaus/Code/train/2_both_based_on_ptz/X_valid.csv", index_col='Unnamed: 0')
X_valid

In [ ]:
X_test = pd.read_csv("D:/Allaus/Code/train/2_both_based_on_ptz/X_test.csv", index_col='Unnamed: 0')
X_test

In [ ]:
y_train = pd.read_csv("D:/Allaus/Code/train/2_both_based_on_ptz/y_train.csv", index_col='Unnamed: 0')
y_train = y_train.Avalanche
y_train

In [ ]:
y_valid = pd.read_csv("D:/Allaus/Code/train/2_both_based_on_ptz/y_valid.csv", index_col='Unnamed: 0')
y_valid = y_valid.Avalanche
y_valid

In [ ]:
y_test = pd.read_csv("D:/Allaus/Code/train/2_both_based_on_ptz/y_test.csv", index_col='Unnamed: 0')
y_test = y_test.Avalanche
y_test

In case I wanted to try without the extra parameters that I created \
(I tried it and the models become worse)

In [ ]:
len(X_train.columns)

In [ ]:
X_train_minus1 = X_train.drop(columns=['SnowCover', 'CriticalRecharge'])
X_train_minus1

In [ ]:
X_valid_minus1 = X_valid.drop(columns=['SnowCover', 'CriticalRecharge'])
X_valid_minus1

In [ ]:
X_test_minus1 = X_test.drop(columns=['SnowCover', 'CriticalRecharge'])
X_test_minus1

Try with even less data

In [ ]:
X_train_minus2 = X_train.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain', 'Rain2D',
       'Rain3D', 'ExtremeRain', 'MonthlyRain', 'Tmin', 'Tmax', 'Tmean',
       'Trange', 'SnowTotal', 'Snow24h', 'Snow48h', 'Snow72h', 'MonthlySnow',
       'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])
X_train_minus2

In [ ]:
X_valid_minus2 = X_valid.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain', 'Rain2D',
       'Rain3D', 'ExtremeRain', 'MonthlyRain', 'Tmin', 'Tmax', 'Tmean',
       'Trange', 'SnowTotal', 'Snow24h', 'Snow48h', 'Snow72h', 'MonthlySnow',
       'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])
X_valid_minus2

In [ ]:
X_test_minus2 = X_test.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain', 'Rain2D',
       'Rain3D', 'ExtremeRain', 'MonthlyRain', 'Tmin', 'Tmax', 'Tmean',
       'Trange', 'SnowTotal', 'Snow24h', 'Snow48h', 'Snow72h', 'MonthlySnow',
       'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])
X_test_minus2

In [ ]:
y_train_minus1 = y_train
y_train_minus2 = y_train
y_valid_minus1 = y_valid
y_valid_minus2 = y_valid
y_test_minus1 = y_test
y_test_minus2 = y_test

In [ ]:
#y_test_minus2.shape

In [ ]:
#y_test.shape

In [ ]:
#y_test_minus1

In [ ]:
frames_X_depl_minus1 = [X_train_minus1, X_valid_minus1]
frames_y_depl_minus1 = [y_train_minus1, y_valid_minus1]

frames_X_depl_minus2 = [X_train_minus2, X_valid_minus2]
frames_y_depl_minus2 = [y_train_minus2, y_valid_minus2]

In [ ]:
X_depl_minus1 = pd.concat(frames_X_depl_minus1)
y_depl_minus1 = pd.concat(frames_y_depl_minus1)

X_depl_minus2 = pd.concat(frames_X_depl_minus2)
y_depl_minus2 = pd.concat(frames_y_depl_minus2)


In [ ]:
y_train_minus1 = y_train.to_numpy().ravel()
y_train_minus2 = y_train.to_numpy().ravel()
y_valid_minus1 = y_valid.to_numpy().ravel()
y_valid_minus2 = y_valid.to_numpy().ravel()
# y_test_minus1 = y_test_minus1.to_numpy().ravel()
# y_test_minus2 = y_test_minus2.to_numpy().ravel()

In [ ]:
y_depl_minus1 = y_depl_minus1.to_numpy().ravel()
y_depl_minus2 = y_depl_minus2.to_numpy().ravel()

**Join training and validation sets for model deployment**

In [ ]:
frames_X = [X_train, X_valid]
frames_y = [y_train, y_valid]

In [ ]:
X_depl = pd.concat(frames_X)
y_depl = pd.concat(frames_y)

In [ ]:
#X_depl.tail()

In [ ]:
y_depl = y_depl.to_numpy()
y_depl

In [ ]:
y_depl = y_depl.ravel()
y_depl

In [ ]:
#y_depl.shape

In [ ]:
y_test = y_test.to_numpy().ravel()


In [ ]:
#y_test.shape

## Model development

### 1) Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.tree import _tree

Build and train initial tree

In [ ]:
clf_dt = DecisionTreeClassifier(random_state=42)
clf_dt = clf_dt.fit(X, y)

Plot trained tree


In [ ]:
clf_dt.classes_

In [ ]:
plt.figure(figsize=(15, 7))
plot_tree(clf_dt,
         filled = True,
         rounded = True,
         class_names = ['Not Susceptible', 'Susceptible'],
         feature_names = X.columns)                 # It is clearly overfitting!!!!!
#plt.savefig("D:/Allaus/Manuscript/pictures_and_figures/DT - overfitted")
plt.show();

Check the performance of the initial model using the VALIDATION SET


Confusion matrix:

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_dt, X_valid_minus2, y_valid_minus2, display_labels=['Not Susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
#plt.title("Confusion Matrix")
plt.show()

Check other performance metrics:

In [ ]:
p_pred = clf_dt.predict_proba(X_valid_minus2)       
y_pred = clf_dt.predict(X_valid_minus2)

In [ ]:
#p_pred

In [ ]:
score = clf_dt.score(X_valid_minus2, y_valid_minus2)
score

In [ ]:
print(metrics.accuracy_score(y_valid_minus2, y_pred))    # same as above cell, different way

Check the performance of the initial model on unseen data (TEST SET)

Confusion matrix:

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_dt, X_test, y_test, display_labels=['Not Susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.show()

Check other performance metrics:

In [ ]:
p_pred = clf_dt.predict_proba(X_test)       
y_pred = clf_dt.predict(X_test)

In [ ]:
score = clf_dt.score(X_test, y_test)     
score                                    # Not amazing accuracy, thus I need to optimize the model
                                         # Here I chose to do pruning by optimizing the alpha parameter instead of
                                         # doing hyperparameter tuning, e.g., with GridSearchCV for all (or many of)
                                         # the parameters that the Decision Tree in sklearn has

In [ ]:
print(metrics.accuracy_score(y_test, y_pred))    # same as above cell, different way

In [ ]:
print(metrics.recall_score(y_test, y_pred, pos_label=0))    

In [ ]:
print(metrics.precision_score(y_test, y_pred, pos_label=1))

Compare the model's accuracy to null accuracy \
\
**Null Accuracy:** the accuracy that could be achieved by always predicting the most frequent class 

In [ ]:
y_valid.value_counts()       # pretty balanced

In [ ]:
# Calculate the percentage of ones
a = y_valid.mean()
a

In [ ]:
# Calculate the percentage of zeros
b = 1 - y_valid.mean()
b

In [ ]:
# Null accuracy for binary classification problems
if a > b:
    print(a)
else:
    print(b)

**Optimization with Cost Complexity Pruning**

Cost Complexity Pruning Part 1: Visualize alpha

In [ ]:
path = clf_dt.cost_complexity_pruning_path(X_train_minus2, y_train_minus2)
ccp_alphas = path.ccp_alphas
ccp_alphas = ccp_alphas[:-1]

clf_dts = []
for ccp_alpha in ccp_alphas:
    clf_dt = DecisionTreeClassifier(random_state = 0, ccp_alpha = ccp_alpha)
    clf_dt.fit(X_train_minus2, y_train_minus2)
    clf_dts.append(clf_dt)

In [ ]:
train_scores = [clf_dt.score(X_train_minus2, y_train_minus2) for clf_dt in clf_dts]
valid_scores = [clf_dt.score(X_valid_minus2, y_valid_minus2) for clf_dt in clf_dts]

fig, ax = plt.subplots()
ax.set_xlabel('alpha')
ax.set_ylabel('accuracy')
ax.set_title('Accuracy vs alpha for training and validation sets')
ax.plot(ccp_alphas, train_scores, marker = 'o', label='training', drawstyle='steps-post')
ax.plot(ccp_alphas, valid_scores, marker = 'o', label='validation', drawstyle='steps-post')
ax.legend()
#plt.savefig("D:/Allaus/Manuscript/pictures_and_figures/DT - alpha vs accuracy")
plt.show();

Cost Complexity Pruning Part 2: Cross validation for finding the best alpha

In [ ]:
clf_dt = DecisionTreeClassifier(random_state=42, ccp_alpha=0.02)      

scores = cross_val_score(clf_dt, X_train_minus2, y_train_minus2, cv=10)
df = pd.DataFrame(data={'tree': range(10), 'accuracy': scores})

df.plot(x='tree', y='accuracy', marker='o', linestyle='--');


In [ ]:
alpha_loop_values = []

for ccp_alpha in ccp_alphas:
    clf_dt = DecisionTreeClassifier(random_state = 0, ccp_alpha = ccp_alpha)
    scores = cross_val_score(clf_dt, X_train_minus2, y_train_minus2, cv=10)
    alpha_loop_values.append([ccp_alpha, np.mean(scores), np.std(scores)])
    
alpha_results = pd.DataFrame(alpha_loop_values, columns=['alpha', 'mean_accuracy', 'std'])

alpha_results.plot(x='alpha',
                   y='mean_accuracy',
                   yerr='std',
                   marker='o',
                   linestyle='--');

In [ ]:
alpha_results[(alpha_results['alpha'] > 0.005)
             & (alpha_results['alpha'] < 0.02)]

In [ ]:
ideal_ccp_alpha = 0.014

Build and train a pruned version of the tree

In [ ]:
X_train

In [ ]:
clf_dt_pruned = DecisionTreeClassifier(random_state=42, ccp_alpha = ideal_ccp_alpha) 
clf_dt_pruned = clf_dt_pruned.fit(X, y)

Check the performance of the pruned tree on the VALIDATION SET 

Confusion matrix:

In [ ]:
plt.figure(figsize=(12, 8))
ConfusionMatrixDisplay.from_estimator(clf_dt_pruned, X_valid_minus2, y_valid_minus2, display_labels=['Not Susceptible', 'Susceptible'], cmap=plt.cm.Blues);
#plt.yticks(rotation = 90)
#plt.tight_layout
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/DT - CM_pruned_2', dpi=300, bbox_inches='tight')
plt.show();
# plt.yticks


Other metrics

In [ ]:
p_pred = clf_dt_pruned.predict_proba(X_valid_minus2)
y_pred = clf_dt_pruned.predict(X_valid_minus2)

In [ ]:
# # Store the predicted probabilities for class 1

# p_aval = p_pred[:,1]
# p_aval

In [ ]:
score = clf_dt_pruned.score(X_valid_minus2, y_valid_minus2)
score

In [ ]:
np.mean(cross_val_score(clf_dt_pruned, X_valid, y_valid, cv=10))

In [ ]:
print(metrics.accuracy_score(y_valid, y_pred))       # same as above, different way

In [ ]:
print(classification_report(y_valid, y_pred))

In [ ]:
print(metrics.recall_score(y_valid, y_pred, pos_label=1))

In [ ]:
print(metrics.precision_score(y_valid, y_pred, pos_label=1))

Check the performance of the pruned tree on the TEST SET (**only at the very end**)

Confusion matrix:

In [ ]:
clf_dt_pruned.classes_

In [ ]:
X_test

In [ ]:
X_depl

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_dt_pruned, X_test_minus2, y_test_minus2, display_labels=['Not Susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/DT - CM test', dpi=300, bbox_inches='tight')
plt.show()


Check other performance metrics

In [ ]:
p_pred = clf_dt_pruned.predict_proba(X_test_minus2)       
y_pred = clf_dt_pruned.predict(X_test_minus2)

In [ ]:
score = clf_dt_pruned.score(X_test_minus2, y_test_minus2)
score

In [ ]:
print(metrics.recall_score(y_test, y_pred, pos_label=1))

In [ ]:
print(metrics.precision_score(y_test, y_pred, pos_label=1))

In [ ]:
np.mean(cross_val_score(clf_dt_pruned, X_test_minus2, y_test_minus2, cv=10))

Plot the pruned tree

In [ ]:
plt.figure(figsize=(15, 7.5))
# plt.savefig('D:/Allaus/Data_analysis/ML/tree.png', dpi=300)
plot_tree(clf_dt_pruned,
         filled = True,
         rounded = True,
         class_names = ['Not Susceptible', 'Susceptible'],
         feature_names = X.columns); 
#plt.savefig("D:/Allaus/Manuscript/pictures_and_figures/DT - for map")
plt.show();

Plot the **rules** of the pruned tree**

In [ ]:
text_representation = tree.export_text(clf_dt_pruned, feature_names=(list(X.columns)), show_weights=True)
print(text_representation)

Visualization with dtreeviz

In [ ]:
viz = dtreeviz(clf_dt_pruned,
               X_valid,
               y_valid,
               target_name = 'Avalanche',
               feature_names = X_train.columns)

In [ ]:
viz

### 2) Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Build the initial model

In [ ]:
clf_rf = RandomForestClassifier(criterion='gini', max_depth=8, min_samples_split=10, random_state=5)
clf_rf.fit(X_train_minus2, y_train_minus2)

In [ ]:
# Feature importances
clf_rf.feature_importances_

In [ ]:
features = X_train.columns
importances=clf_rf.feature_importances_
indices=np.argsort(importances)

plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color='g', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel("Relative importance")
plt.show()

Check the accuracy of the initial model (VALIDATION SET)

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_rf, X_valid_minus2, y_valid_minus2, display_labels=['Not Susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.show()

In [ ]:
y_pred = clf_rf.predict(X_valid_minus2)

In [ ]:
accuracy_score(y_valid_minus2, y_pred)

In [ ]:
# This is an alternative to the train-valid-test split, that shows us the variance of the accuracy 
# by bootstrapping the X with 10-fold cross-validation

np.mean(cross_val_score(clf_rf, X_valid_minus2, y_valid_minus2, cv=10))

In [ ]:
print(classification_report(y_valid_minus2, y_pred))

Check the accuracy of the initial model (TEST SET)

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_rf, X_test, y_test, display_labels=['Susceptible', 'Not susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.show()

In [ ]:
y_pred = clf_rf.predict(X_test)
accuracy_score(y_test, y_pred)          

In [ ]:
print(metrics.classification_report(y_test, y_pred))

Optimization

**Alternative A:** RandomizedSearchCV() to make the optimization quicker

In [ ]:
model = RandomForestClassifier(n_jobs = -1)

parameters = {'min_samples_split': sp_randInt(2, 8),
              'criterion': ('gini', 'entropy'),
              'n_estimators': sp_randInt(10, 200),
              'max_depth': sp_randInt(5, 8)}

rf_grid_randm = RandomizedSearchCV(estimator=model, param_distributions=parameters, cv=10, n_iter=100, n_jobs=-1)

rf_grid_randm.fit(X_train, y_train)

print(rf_grid_randm.best_estimator_)
print(rf_grid_randm.best_score_)
print(rf_grid_randm.best_params_)           

**Alterntive B:** GridSearchCV(), more thorough but also slower

In [ ]:
parameters = {'n_estimators': (100, 200),
              'criterion': ('gini', 'entropy'),
              'max_depth': (3, 5, 7),
              'max_features': ['sqrt'],
              'min_samples_split': (2, 4, 6, 8),
              'min_weight_fraction_leaf': (0.0, 0.1, 0.2, 0.3)
}

In [ ]:
rf_grid = GridSearchCV(RandomForestClassifier(n_jobs = -1, oob_score = False), param_grid = parameters, cv=10, verbose = True)

In [ ]:
rf_grid_model = rf_grid.fit(X_train, y_train)

In [ ]:
rf_grid_model.best_estimator_         

In [ ]:
rf_grid_model.best_score_

Build the model with the best estimates

In [ ]:
clf_rf = RandomForestClassifier(criterion='entropy', max_depth=7, min_samples_split=4, n_estimators=139,
                       n_jobs=-1, random_state=42)
clf_rf.fit(X_train, y_train)


In [ ]:
plt.figure(figsize=(12, 8))
ConfusionMatrixDisplay.from_estimator(clf_rf, X_valid_minus2, y_valid_minus2, display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/RF optimized - CM', dpi=300, bbox_inches='tight')
plt.show();


In [ ]:
#clf_rf.estimators_

In [ ]:
len(clf_rf.estimators_)      # of course

In [ ]:
max = 0
idx_max = 0
min = 1
idx_min = 0
for i in range(len(clf_rf.estimators_)):
    if clf_rf.estimators_[i].score(X_valid, y_valid) > max:
        max = clf_rf.estimators_[i].score(X_valid, y_valid)
        idx_max = i
    elif clf_rf.estimators_[i].score(X_valid, y_valid) < min:
        min = clf_rf.estimators_[i].score(X_valid, y_valid)
        idx_min = i

print(idx_max)
print(max)
print(idx_min)
print(min)

In [ ]:
clf_rf.estimators_[43].score(X_valid.values, y_valid.values)         # check for the best estimator

In [ ]:
y_pred = clf_rf.predict(X_valid_minus2)

In [ ]:
accuracy_score(y_valid_minus2, y_pred)

In [ ]:
clf_rf.score(X_valid, y_valid)

In [ ]:
np.mean(cross_val_score(clf_rf, X_valid_minus2, y_valid_minus2, cv=10))

In [ ]:
print(metrics.recall_score(y_valid, y_pred, pos_label=1))

In [ ]:
print(metrics.precision_score(y_valid, y_pred, pos_label=1))

In [ ]:
plt.figure(figsize=(15, 7.5))
# plt.savefig('D:/Allaus/Data_analysis/ML/tree.png', dpi=300)
plot_tree(clf_rf[43],
         filled = True,
         rounded = True,
         class_names = ['Not susceptible', 'Susceptible'],
         feature_names = X_train.columns);                     

In [ ]:
X_train.columns

In [ ]:
lista = X_train.columns.values.tolist()
#lista

In [ ]:
text_repr = tree.export_text(clf_rf.estimators_[43], feature_names = lista, show_weights=True)  
print(text_repr)  

In [ ]:
features = X_train.columns
importances=clf_rf.feature_importances_
indices=np.argsort(importances)

plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color='g', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel("Relative importance")
plt.savefig("D:/Allaus/Manuscript/pictures_and_figures/RF - feature importance", dpi=300, bbox_inches='tight')
plt.show()

Performance on TEST SET

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_rf, X_test_minus2, y_test_minus2, display_labels=['Susceptible', 'Not susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/RF - CM test', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
y_pred = clf_rf.predict(X_test_minus2)

In [ ]:
print(classification_report(y_test_minus2, y_pred))

In [ ]:
np.mean(cross_val_score(clf_rf, X_test_minus2, y_test_minus2, cv=10))

### 3) Extra Trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

Build the initial model

In [ ]:
clf_et = ExtraTreesClassifier(criterion='entropy', random_state=5)
clf_et.fit(X_train, y_train)

In [ ]:
# Feature importances
clf_et.feature_importances_

In [ ]:
features = X_train.columns
importances=clf_et.feature_importances_
indices=np.argsort(importances)

plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color='g', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel("Relative importance")
plt.show()

Check the accuracy of the initial model (VALIDATION SET)

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_et, X_valid, y_valid, display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.show()

In [ ]:
y_pred = clf_et.predict(X_valid)

In [ ]:
accuracy_score(y_valid, y_pred)

In [ ]:
cross_val_score(clf_et, X_valid, y_valid, cv=10) 

Optimization

In [ ]:
# # Alternative A: Very slow, I'm not going to use it

# parameters = {'n_estimators': (10, 30, 50, 70, 90, 100),
#               'criterion': ('gini', 'entropy', 'log_loss'),
#               'max_depth': (3, 5, 7, 9),
#               'max_features': ['sqrt'],
#               'min_samples_split': (2, 4, 6),
#               'min_weight_fraction_leaf': (0.0, 0.1, 0.2, 0.3)
# }

# et_grid = GridSearchCV(ExtraTreesClassifier(n_jobs = -1, oob_score = False), param_grid = parameters, cv=10, verbose = True)


In [ ]:
# et_grid_model = et_grid.fit(X_train, y_train)
# et_grid_model.best_estimator_ 
# et_grid_model.best_score_
# et_grid_model.best_params_

In [ ]:
# Alternative B: A looot faster

model = ExtraTreesClassifier(n_jobs = -1)

parameters = {'min_samples_split': sp_randInt(2, 10),
              'criterion': ('gini', 'entropy', 'log_loss'),
              'min_weight_fraction_leaf': sp_randFloat(0.0, 0.5),
              'n_estimators': sp_randInt(10, 100),
              'max_depth': sp_randInt(5, 7)}

et_grid_randm = RandomizedSearchCV(estimator=model, param_distributions=parameters, cv=10, n_iter=100, n_jobs=-1)

et_grid_randm.fit(X_train, y_train)

print(et_grid_randm.best_estimator_)
print(et_grid_randm.best_score_)
print(et_grid_randm.best_params_)           # I'm gonna use this, cause it's a lot faster

Build the model with the best estimates

In [ ]:
clf_et = ExtraTreesClassifier(criterion='gini', max_depth=9, min_samples_split=4, max_features = 'sqrt',
                     min_weight_fraction_leaf=0,
                     n_estimators=90, n_jobs=-1)
clf_et.fit(X_train, y_train)

In [ ]:
#clf_et.estimators_

In [ ]:
max = 0
idx_max = 0
min = 1
idx_min = 0
for i in range(len(clf_et.estimators_)):
    if clf_et.estimators_[i].score(X_valid, y_valid) > max:
        max = clf_et.estimators_[i].score(X_valid, y_valid)
        idx_max = i
    elif clf_et.estimators_[i].score(X_valid, y_valid) < min:
        min = clf_et.estimators_[i].score(X_valid, y_valid)
        idx_min = i

print(idx_max)
print(max)
print(idx_min)
print(min)

In [ ]:
plt.figure(figsize=(15, 7.5))
# plt.savefig('D:/Allaus/Data_analysis/ML/tree.png', dpi=300)
plot_tree(clf_et[4],
         filled = True,
         rounded = True,
         class_names = ['Not susceptible', 'Susceptible'],
         feature_names = X_train.columns);                     

Performance on VALIDATION SET

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_et, X_valid, y_valid, display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.show()           # Worse than before optimization......


In [ ]:
y_pred = clf_et.predict(X_valid)

In [ ]:
accuracy_score(y_valid, y_pred)

In [ ]:
clf_et.score(X_valid, y_valid)    # Bad accuracy, I will probably use it only for feature selection

In [ ]:
print(metrics.recall_score(y_valid, y_pred, pos_label=1))

In [ ]:
print(metrics.precision_score(y_valid, y_pred, pos_label=1))

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_et, X_test, y_test, display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.show()           

In [ ]:
y_pred = clf_et.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred))

### 4) AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

---|--- Short way ---|---

In [ ]:
# Alternative A: Very slow

ada = AdaBoostClassifier()
search_grid = {'n_estimators': [100, 150, 200], 'learning_rate': [0.1, 0.03, 0.5]}
search = GridSearchCV(estimator = ada, param_grid = search_grid, scoring = 'accuracy', n_jobs = -1, cv = 10)

In [ ]:
search.fit(X_train, y_train)            # It takes toooo long
print(search.best_params_)
print(search.best_score_)

In [ ]:
# Alternative B: A looot faster

model = AdaBoostClassifier()

parameters = {'learning_rate': (0.1, 0.3, 0.5),
              'n_estimators': sp_randInt(10, 500)}

ada_grid_randm = RandomizedSearchCV(estimator=model, param_distributions=parameters, cv=10, n_iter=100, n_jobs=-1)

ada_grid_randm.fit(X_train_minus1, y_train_minus1)

print(ada_grid_randm.best_estimator_)
print(ada_grid_randm.best_score_)
print(ada_grid_randm.best_params_)           # I'm gonna use this, cause it's faster (but still very slow)

Build of model with otimized parameters

In [ ]:
ada = AdaBoostClassifier(learning_rate=0.5, n_estimators=425, random_state=42)
ada.fit(X_train, y_train)

Performance based on VALIDATION SET

In [ ]:
ConfusionMatrixDisplay.from_estimator(ada, X_valid_minus2, y_valid_minus2, display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/ada optimized - CM', dpi=300, bbox_inches='tight')
plt.show();


In [ ]:
y_pred = ada.predict(X_valid_minus2)

In [ ]:
accuracy_score(y_valid_minus2, y_pred)

In [ ]:
score = np.mean(cross_val_score(ada, X_valid_minus2, y_valid_minus2, scoring='accuracy', cv=10, n_jobs=-1))  
score

In [ ]:
print(metrics.recall_score(y_valid, y_pred, pos_label=1))

In [ ]:
print(metrics.precision_score(y_valid, y_pred, pos_label=1))

In [ ]:
ConfusionMatrixDisplay.from_estimator(ada, X_test_minus2, y_test_minus2, display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/ada - CM test', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
y_pred = ada.predict(X_test_minus2)

In [ ]:
print(metrics.classification_report(y_test_minus2, y_pred))

In [ ]:
score = np.mean(cross_val_score(ada, X_test_minus1, y_test_minus1, scoring='accuracy', cv=10, n_jobs=-1))  
score

---|--- Longer way ---|---

In [ ]:
class DecisionStump:
    
    def __init__(self):
        self.polarity = 1
        self. geature_idx = None
        self.threshold = None
        self.alpha = None
        
    def predict(self, X):
        n_samples = X.shape[0]
        X_column = X.iloc[:, self.feature_idx]
        
        predictions = np.ones(n_samples)
        if self.polarity ==1:
            predictions[X_column < self.threshold] = -1
        else:
            predictions[X_column > self.threshold] = -1
        return predictions

class Adaboost:
    
    def __init__(self, n_clf=5):
        self.n_clf = n_clf
        
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        # init weights
        w = np.full(n_samples, (1 / n_samples))
        
        self.clfs = []
        for _ in range(self.n_clf):
            clf = DecisionStump()
            
            min_error = float('inf')
            
            for feature_i in range(n_features):
                X_column = X.iloc[:, feature_i]
                thresholds = np.unique(X_column)
                
                for threshold in thresholds:
                    p = 1
                    predictions = np.ones(n_samples)
                    predictions[X_column < threshold] = -1
                    
                    missclassified = w[y != predictions]
                    error = sum(missclassified)
                    
                    if error > 0.5:
                        error = 1 - error
                        p = -1
                    if error < min_error:
                        min_error = error
                        clf.polarity = p
                        clf.threshold = threshold
                        clf.feature_idx = feature_i
            
            EPS = 1e-10
            clf.alpha = 0.5 * np.log((1 - min_error) / (min_error + EPS))
            
            predictions = clf.predict(X)
            
            w *= np.exp(-clf.alpha * y * predictions)
            w /= np.sum(w)
            
            self.clfs.append(clf)
            
    
    def predict(self, X):
        clf_preds = [clf.alpha * clf.predict(X) for clf in self.clfs]
        y_pred = np.sum(clf_preds, axis=0)
        y_pred = np.sign(y_pred)
        return y_pred


In [ ]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [ ]:
y_train.unique()

In [ ]:
y_train[y_train == 0] = -1
y_train.unique()

In [ ]:
y_valid.unique()

In [ ]:
y_valid[y_valid == 0] = -1
y_valid.unique()

In [ ]:
y_test.unique()

In [ ]:
y_test[y_test == 0] = -1
y_test.unique()

In [ ]:
# Adaboost classification with 500 weak classifiers   It takes a loooong time to run, but has good accuracy probably
clf_ada = Adaboost(n_clf=500)
clf_ada.fit(X_train, y_train)

Performance based on VALIDATION SET

In [ ]:
y_pred = clf_ada.predict(X_valid)

acc = accuracy(y_valid, y_pred)
print("Accuracy:", acc)

### 5) Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

Parameter optimization

In [ ]:
# ROUND 1: Optimization of hyperparamaters with cross validation
param_grid = {
    'max_depth': [1, 3, 5],
    'learning_rate': [0.1, 0.3, 0.5],
    'subsample': [0.5, 0.75, 1],
    'random_state': [1],
    'n_estimators': [100, 500]
}

optimal_params = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=param_grid, scoring='roc_auc', 
                              verbose=0, n_jobs=-1, cv=10)

optimal_params.fit(X_train_minus1, y_train_minus1)

print(optimal_params.best_params_)

In [ ]:
# ROUND 2      # It takes a long time to run!!!
               # It is the optimization of the n_estimators that takes so long. The more the merrier (until some point
               # that my computer is not powerful enough to reach.)
               # That's why I can just set the maximum trees that I want to be created.
               # Or I can try the RandomizedSearchCV().
                
param_grid = {
    'max_depth': [5],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.75],
    'random_state': [1],
    'n_estimators': [500]
}

optimal_params = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=param_grid, scoring='roc_auc', 
                              verbose=0, n_jobs=-1, cv=10)

optimal_params.fit(X_train_minus1, y_train_minus1)

print(optimal_params.best_params_)

Building and training of the model with the optimized parameters

In [ ]:
GBC = GradientBoostingClassifier(max_depth=5, learning_rate=0.1, subsample=0.75, random_state=1, n_estimators=1000)
GBC.fit(X_train, y_train)

**Confusion matrix** using the VALIDATION SET

In [ ]:
ConfusionMatrixDisplay.from_estimator(GBC, X_valid_minus2, y_valid_minus2, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/gradient boost optimized - CM', dpi=300, bbox_inches='tight')
plt.show();


In [ ]:
y_pred = GBC.predict(X_valid)

In [ ]:
accuracy_score(y_valid, y_pred)

In [ ]:
score = np.mean(cross_val_score(GBC, X_valid, y_valid, scoring='accuracy', cv=10, n_jobs=-1))  
score

In [ ]:
print(metrics.recall_score(y_valid, y_pred, pos_label=1))

In [ ]:
print(metrics.precision_score(y_valid, y_pred, pos_label=1))

In [ ]:
ConfusionMatrixDisplay.from_estimator(GBC, X_test_minus2, y_test_minus2, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/gradient boost - CM test', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
y_pred = GBC.predict(X_test_minus2)

In [ ]:
print(metrics.classification_report(y_test_minus2, y_pred))

In [ ]:
score = np.mean(cross_val_score(GBC, X_test_minus1, y_test_minus1, scoring='accuracy', cv=10, n_jobs=-1))  
score

Other metrics

In [ ]:
y_train_pred = GBC.predict_proba(X_train)[:,1]
y_valid_pred = GBC.predict_proba(X_valid)[:,1]
#y_test_pred = GBC.predict_proba(X_test)[:,1]

print("AUC Train: {:.4f}\nAUC Valid: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                                      roc_auc_score(y_valid, y_valid_pred)))

# print("AUC Train: {:.4f}\nAUC Valid: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
#                                                                       roc_auc_score(y_valid, y_valid_pred),
#                                                                       roc_auc_score(y_test, y_test_pred)))

In [ ]:
l = [tree for tree in GBC.staged_predict_proba(X_train)]
trees = np.stack(l)
trees.shape

In [ ]:
y_train_pred_trees = np.stack([tree for tree in GBC.staged_predict_proba(X_train)])[:,:,1]
y_valid_pred_trees = np.stack([tree for tree in GBC.staged_predict_proba(X_valid)])[:,:,1]
#y_test_pred_trees = np.stack([tree for tree in GBC.staged_predict_proba(X_test)])[:,:,1]

y_train_pred_trees.shape, y_valid_pred_trees.shape#, y_test_pred_trees.shape

In [ ]:
# y_train_pred_trees = np.stack(list(GBC.staged_predict_proba(X_train)))[:,:,1]  # for some reason it doesn't work now
# y_valid_pred_trees = np.stack(list(GBC.staged_predict_proba(X_valid)))[:,:,1]
# y_test_pred_trees = np.stack(list(GBC.staged_predict_proba(X_test)))[:,:,1]

# y_train_pred_trees.shape, y_valid_pred_trees.shape, y_test_pred_trees.shape

In [ ]:
auc_train_trees = [roc_auc_score(y_train, y_pred) for y_pred in y_train_pred_trees]
auc_valid_trees = [roc_auc_score(y_valid, y_pred) for y_pred in y_valid_pred_trees]
#auc_test_trees = [roc_auc_score(y_test, y_pred) for y_pred in y_test_pred_trees]

Plot the curves

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(auc_train_trees, label='Train Data')
plt.plot(auc_valid_trees, label='Validation Data')
#plt.plot(auc_test_trees, label='Test Data')

plt.title('AUC vs Number of Trees')
plt.ylabel('AUC')
plt.xlabel('Number of Trees')
plt.legend()

plt.savefig("D:/Allaus/Manuscript/pictures_and_figures/gradient boost - number of trees")
plt.show();

Importance of features

In [ ]:
pd.DataFrame({"Variable_Name":X_train.columns,
              "Importance":GBC.feature_importances_}).sort_values('Importance', ascending=False)



In [ ]:
# Alternative: Adding colors to the data frame

feat = pd.DataFrame({"Variable_Name":X_train.columns, "Importance":GBC.feature_importances_})
cm = sns.light_palette("green", as_cmap=True)
s = feat.style.background_gradient(cmap=cm)
s

In [ ]:
features = X_train.columns
importances=GBC.feature_importances_
indices=np.argsort(importances)

#plt.title("Feature Importances")
plt.barh(range(len(indices)), importances[indices], color='y', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.ylabel("Feature")
plt.xlabel("Relative importance")
plt.savefig("D:/Allaus/Manuscript/pictures_and_figures/gradient boost - feature importance", dpi=300, bbox_inches='tight')
plt.show();

### 6) XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', seed=42, early_stopping_rounds=10, 
                            eval_metric='aucpr', random_state=42)
clf_xgb.fit(X_train,
            y_train,
            verbose=True,
            eval_set=[(X_valid, y_valid)])


Confusion matrix using the VALIDATION SET

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_xgb, X_valid, y_valid, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.show()  


Hyperparameter tuning

In [ ]:
# ROUND 1: Optimization of hyperparamaters with cross validation 
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.3, 0.5],
    'gamma': [0, 0.25, 1],
    'reg_lambda': [0, 1, 10],
    'scale_pos_weight': [1, 3, 5]
}

optimal_params = GridSearchCV(estimator=xgb.XGBClassifier(objective='binary:logistic', seed=42,
                                                          early_stopping_rounds=10, 
                                                          eval_metric='auc',
                                                          #subsample=0.9, 
                                                          #colsample_bytree=0.5
                                                         ),
                              param_grid=param_grid,
                              scoring='roc_auc',
                              verbose=0,  # if you want to see what Grid Search is doing you put 2 instead of 0
                              n_jobs=-1, 
                              cv=10)

optimal_params.fit(X_train_minus1,
                   y_train_minus1,
                   eval_set=[(X_valid_minus1, y_valid_minus1)],
                   verbose=False)

print(optimal_params.best_params_)

In [ ]:
# ROUND 2
param_grid = {
    'max_depth': [3],
    'learning_rate': [0,5, 0.7, 0.9],
    'gamma': [0],
    'reg_lambda': [1],
    'scale_pos_weight': [1]
}

optimal_params = GridSearchCV(estimator=xgb.XGBClassifier(objective='binary:logistic', seed=42,
                                                          early_stopping_rounds=10, 
                                                          eval_metric='auc',
                                                          #subsample=0.9, 
                                                          #colsample_bytree=0.5
                                                         ),
                              param_grid=param_grid,
                              scoring='roc_auc',
                              verbose=0,  # if you want to see what Grid Search is doing you put 2 instead of 0
                              n_jobs=-1, 
                              cv=10)

optimal_params.fit(X_train_minus1,
                   y_train_minus1,
                   eval_set=[(X_valid_minus1, y_valid_minus1)],
                   verbose=False)

print(optimal_params.best_params_)

Building the model with the optimized parameters

In [ ]:
# Now that we have optimised the hyperparameters, we can build the final XGBoost model
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', 
                            seed=42, 
                            #early_stopping_rounds=10, 
                            #eval_metric='aucpr', 
                            gamma=0, 
                            learning_rate=0.7, 
                            max_depth=3, 
                            reg_lambda=1, 
                            scale_pos_weight=1,
                            #subsample=0.9, 
                            #colsample_bytree=0.5
                           )

clf_xgb.fit(X_depl_minus1, 
            y_depl_minus1,
            verbose=True)#,
            #eval_set=[(X_test, y_test)])

Confusion matrix using the VALIDATION SET

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_xgb, X_valid, y_valid, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/xgboost - CM', dpi=300, bbox_inches='tight')
plt.show();


In [ ]:
y_pred = clf_xgb.predict(X_valid)


In [ ]:
accuracy_score(y_valid, y_pred)

In [ ]:
print(metrics.recall_score(y_valid, y_pred, pos_label=1))

In [ ]:
print(metrics.precision_score(y_valid, y_pred, pos_label=1))

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_xgb, X_test, y_test, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/xgboost - CM test', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
y_pred = clf_xgb.predict(X_depl)

In [ ]:
print(metrics.classification_report(y_depl, y_pred))

In [ ]:
inputs = pd.DataFrame(X_test_minus2).reset_index()
inputs

Cross validation

In [ ]:
acc_per_fold = []

# Merge inputs and targets
# inputs = np.concatenate((X_depl, X_test), axis=0)
# targets = np.concatenate((y_depl, y_test), axis=0)
inputs = pd.DataFrame(X_test_minus1).reset_index().drop(columns=["index"])
targets = pd.DataFrame(y_test_minus1).reset_index().drop(columns=["index"])

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    

    # Fit data to model
    clf_xgb = xgb.XGBClassifier(objective='binary:logistic', 
                                seed=42, 
                                early_stopping_rounds=10, 
                                eval_metric='aucpr', 
                                gamma=0, 
                                learning_rate=0.7, 
                                max_depth=3, 
                                reg_lambda=1, 
                                scale_pos_weight=1,
                                #subsample=0.9, 
                                #colsample_bytree=0.5
                               )

    clf_xgb.fit(X_depl_minus1, 
                y_depl_minus1,
                verbose=True,
                eval_set=[(inputs.loc[train, :], targets.loc[train, :])])

    # Generate generalization metrics
    y_pred = clf_xgb.predict(inputs.loc[test, :])
    score = accuracy_score(targets.loc[test, :], y_pred)
    print(f'Score for fold {fold_no}: {score*100}%')
    acc_per_fold.append(score * 100)

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
acc_per_fold

Plot one (the first) tree

This way we can get a first idea of the parameters, in order to know better how to optimize them next

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', 
                            seed=42, 
                            early_stopping_rounds=10, 
                            eval_metric='aucpr', 
                            gamma=0.25, 
                            learning_rate=0.3, 
                            max_depth=5, 
                            reg_lambda=1, 
                            scale_pos_weight=1,
                            #subsample=0.9, 
                            #colsample_bytree=0.5
                            n_estimators=1)

clf_xgb.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)])

bst = clf_xgb.get_booster()
for importance_type in ('weight', 'gain', 'cover', 'total_gain', 'total_cover'):
    print('%s: '% importance_type, bst.get_score(importance_type=importance_type))

node_params = {'shape': 'box',
               'style': 'filled, rounded',
               'fillcolor': '#78cbe',
              }
leaf_params = {'shape': 'box',
               'style': 'filled',
               'fillcolor': '#e48038'}

xgb.to_graphviz(clf_xgb, num_trees=0, size='10, 10', condition_node_params=node_params, leaf_node_params=leaf_params)

# # if you want to save the figure
# graph_data = xgb.to_graphviz(clf_xgb, num_trees=0, size='10, 10',
#                              condition_node_params=node_params, leaf_node_params=leaf_params)
# graph_data.view(filename='xgboost_model')  # it saves a pdf

In [ ]:
acc_per_fold

**Multi-collinearity test**

The higher the value of VIF the higher correlation between this variable and the rest.

In [ ]:
X_train

In [ ]:
# Compute VIF data for each independent variable
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif = pd.DataFrame()
vif["features"] = X_train.columns
vif["vif_Factor"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
vif                       

In [ ]:
vif.sort_values(by='vif_Factor', ascending=False)

In [ ]:
X_train.Rain.corr(X_train.Snow24h)

**Scaling / Standardization of the data**

The radial Basis Function (RBF), **which is the default kernel of the SVC object of the sklearn package**, assumes that the data are centered and scaled. In other words, each column should have a mean value = 0 and a standard deviation = 1. So, we need to do this to the train, validation and test datasets. It is also necessary for the Linear Regression and the NN.

NOTE 1: We will first split the data into train, valid, test and then scale them separately in order to avoid Data Leakage. **Data Leakage** occurs when information about the training dataset corrupts or influences the validation and testing datasets.

NOTE 2: Logistic regression and SVM with a linear kernel have similar performance but depending on your features, one may be more efficient than the other.

NOTE 3: It is important to standardize the validation and test set using the mean and std of the train set and not calculate it separately for all datasets. The reason is that a true test of the built model is how it does on unseen data using the parameters to learn from the training set.

**a) Prepare special dataset for SVM and Logistic Regression that suffer from multicollinearity**

In [ ]:
X_train

In [ ]:
X_train_mc = X_train.drop(columns=['Rain2D', 'Rain3D', 'Snow48h', 'Snow72h', 
                                   'Tmax', 'Tmin', 'Trange']).copy()
X_train_mc

In [ ]:
y_train_mc = y_train.copy()

In [ ]:
y_train_mc.shape

In [ ]:
X_valid

In [ ]:
X_valid_mc = X_valid.drop(columns=['Rain2D', 'Rain3D', 'Snow48h', 'Snow72h', 
                                   'Tmax', 'Tmin', 'Trange']).copy()
X_valid_mc

In [ ]:
y_valid_mc = y_valid.copy()

In [ ]:
y_valid_mc.shape

In [ ]:
X_test

In [ ]:
X_test_mc = X_test.drop(columns=['Rain2D', 'Rain3D', 'Snow48h', 'Snow72h', 'Tmax', 'Tmin', 'Trange']).copy()
X_test_mc

In [ ]:
y_test_mc = y_test.copy()

In [ ]:
y_test_mc.shape

**b) Prepare datasets for NN that needs the data to be scaled**

In [ ]:
X_train.columns

In [ ]:
len(X_train.columns)

In [ ]:
X_train_nn = X_train.copy()
X_train

In [ ]:
X_valid_nn = X_valid.copy()
X_valid_nn

In [ ]:
X_test_nn = X_test.copy()
X_test_nn

In [ ]:
y_train_nn = y_train.copy()

In [ ]:
y_valid_nn = y_valid.copy()

In [ ]:
y_test_nn = y_test.copy()

In [ ]:
y_train_nn

In [ ]:
mean_nn = X_train_nn[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'Rain2D', 'Rain3D', 'Tmin', 'Tmax', 'Tmean', 'Trange', 'SnowTotal', 'MonthlyRain', 'MonthlySnow',
                'Snow24h', 'Snow48h', 'Snow72h', 'WindSpeedMean', 'WindSpeedMax', 'ExtremeRain']].mean(axis=0)
mean_nn

In [ ]:
std_nn = X_train_nn[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'Rain2D', 'Rain3D', 'Tmin', 'Tmax', 'Tmean', 'Trange', 'SnowTotal', 'MonthlyRain', 'MonthlySnow',
                'Snow24h', 'Snow48h', 'Snow72h', 'WindSpeedMean', 'WindSpeedMax', 'ExtremeRain']].std(axis=0)
std_nn

In [ ]:
X_train_nn[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'Rain2D', 'Rain3D', 'Tmin', 'Tmax', 'Tmean', 'Trange', 'SnowTotal', 'MonthlyRain', 'MonthlySnow',
                'Snow24h', 'Snow48h', 'Snow72h', 'WindSpeedMean', 'WindSpeedMax', 'ExtremeRain']] -= mean_nn
X_train_nn[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'Rain2D', 'Rain3D', 'Tmin', 'Tmax', 'Tmean', 'Trange', 'SnowTotal', 'MonthlyRain', 'MonthlySnow',
                'Snow24h', 'Snow48h', 'Snow72h', 'WindSpeedMean', 'WindSpeedMax', 'ExtremeRain']] /= std_nn

X_valid_nn[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'Rain2D', 'Rain3D', 'Tmin', 'Tmax', 'Tmean', 'Trange', 'SnowTotal', 'MonthlyRain', 'MonthlySnow',
                'Snow24h', 'Snow48h', 'Snow72h', 'WindSpeedMean', 'WindSpeedMax', 'ExtremeRain']] -= mean_nn
X_valid_nn[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'Rain2D', 'Rain3D', 'Tmin', 'Tmax', 'Tmean', 'Trange', 'SnowTotal', 'MonthlyRain', 'MonthlySnow',
                'Snow24h', 'Snow48h', 'Snow72h', 'WindSpeedMean', 'WindSpeedMax', 'ExtremeRain']] /= std_nn

X_test_nn[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'Rain2D', 'Rain3D', 'Tmin', 'Tmax', 'Tmean', 'Trange', 'SnowTotal', 'MonthlyRain', 'MonthlySnow',
                'Snow24h', 'Snow48h', 'Snow72h', 'WindSpeedMean', 'WindSpeedMax', 'ExtremeRain']] -= mean_nn
X_test_nn[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'Rain2D', 'Rain3D', 'Tmin', 'Tmax', 'Tmean', 'Trange', 'SnowTotal', 'MonthlyRain', 'MonthlySnow',
                'Snow24h', 'Snow48h', 'Snow72h', 'WindSpeedMean', 'WindSpeedMax', 'ExtremeRain']] /= std_nn

In [ ]:
X_train_nn

**---------------------------------------------------------------------------------------------------------------------**

In [ ]:
X_train_mc.columns

In [ ]:
X_train_mc

In [ ]:
len(X_train_mc.columns)

In [ ]:
mean_mc = X_train_mc[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'MonthlySnow', 'Snow24h', 
                'WindSpeedMean', 'WindSpeedMax']].mean(axis=0)
mean_mc

In [ ]:
std_mc = X_train_mc[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'MonthlySnow', 'Snow24h', 
                'WindSpeedMean', 'WindSpeedMax']].std(axis=0)
std_mc

In [ ]:
X_train_mc[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'MonthlySnow', 'Snow24h', 
                'WindSpeedMean', 'WindSpeedMax']] -= mean_mc
X_train_mc[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'MonthlySnow', 'Snow24h', 
                'WindSpeedMean', 'WindSpeedMax']] /= std_mc

X_valid_mc[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'MonthlySnow', 'Snow24h', 
                'WindSpeedMean', 'WindSpeedMax']] -= mean_mc
X_valid_mc[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'MonthlySnow', 'Snow24h', 
                'WindSpeedMean', 'WindSpeedMax']] /= std_mc

X_test_mc[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'MonthlySnow', 'Snow24h', 
                'WindSpeedMean', 'WindSpeedMax']] -= mean_mc
X_test_mc[['Elevation', 'Slope', 'CurvaturePlan', 'CurvatureProf', 'SnowCover', 'CriticalRecharge', 
                'Rain', 'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'MonthlySnow', 'Snow24h', 
                'WindSpeedMean', 'WindSpeedMax']] /= std_mc

In [ ]:
X_train.dtypes

In [ ]:
X_train_mc.dtypes

In [ ]:
X_train_nn.dtypes

In [ ]:
# Test to see that the binary variables did not get affected by the standardization
plt.hist(X_train_nn.Forest)
plt.show()

In [ ]:
# The same for the target
plt.hist(y_train_mc)
plt.show()

In [ ]:
X_train_mc.shape, X_valid_mc.shape, X_test_mc.shape

In [ ]:
# from sklearn.preprocessing import scale

In [ ]:
# X_train_mc_scaled = scale(X_train_mc)
# X_valid_mc_scaled = scale(X_valid_mc)
# X_test_mc_scaled = scale(X_test_mc)

In [ ]:
# X_train_mc_scaled[4]

**Build the deployment datasets for Log Reg, SVM & NN**

In [ ]:
frames_X_nn_depl = [X_train_nn, X_valid_nn]
frames_y_nn_depl = [y_train_nn, y_valid_nn]

In [ ]:
frames_X_mc_depl = [X_train_mc, X_valid_mc]
frames_y_mc_depl = [y_train_mc, y_valid_mc]

In [ ]:
X_depl_nn = pd.concat(frames_X_nn_depl)
y_depl_nn = pd.concat(frames_y_nn_depl)

In [ ]:
X_depl_mc = pd.concat(frames_X_mc_depl)
y_depl_mc = pd.concat(frames_y_mc_depl)

In [ ]:
X_depl_mc

In [ ]:
y_depl_mc = y_depl_mc.to_numpy().ravel()
y_depl_nn = y_depl_nn.to_numpy().ravel()

In [ ]:
y_depl_mc.shape, y_depl_nn.shape

In [ ]:
y_test_mc = y_test_mc.to_numpy().ravel()
y_test_nn = y_test_nn.to_numpy().ravel()

In [ ]:
y_test_mc.shape, y_test_nn.shape

**Other versions of the dataset**

In [ ]:
X_train_mc.columns

In [ ]:
X_train_mc_minus1 = X_train_mc.drop(columns=['SnowCover', 'CriticalRecharge'])
X_valid_mc_minus1 = X_valid_mc.drop(columns=['SnowCover', 'CriticalRecharge'])
X_test_mc_minus1 = X_test_mc.drop(columns=['SnowCover', 'CriticalRecharge'])

X_train_mc_minus2 = X_train_mc.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain',
       'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'Snow24h',
       'MonthlySnow', 'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])
X_valid_mc_minus2 = X_valid_mc.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain',
       'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'Snow24h',
       'MonthlySnow', 'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])
X_test_mc_minus2 = X_test_mc.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain',
       'ExtremeRain', 'MonthlyRain', 'Tmean', 'SnowTotal', 'Snow24h',
       'MonthlySnow', 'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])


In [ ]:
y_train_mc_minus1 = y_train_mc
y_train_mc_minus2 = y_train_mc
y_valid_mc_minus1 = y_valid_mc
y_valid_mc_minus2 = y_valid_mc
y_test_mc_minus1 = y_test_mc
y_test_mc_minus2 = y_test_mc

In [ ]:
frames_X_mc_depl_minus1 = [X_train_mc_minus1, X_valid_mc_minus1]
frames_y_mc_depl_minus1 = [y_train_mc_minus1, y_valid_mc_minus1]

frames_X_mc_depl_minus2 = [X_train_mc_minus2, X_valid_mc_minus2]
frames_y_mc_depl_minus2 = [y_train_mc_minus2, y_valid_mc_minus2]

In [ ]:
X_depl_mc_minus1 = pd.concat(frames_X_mc_depl_minus1)
y_depl_mc_minus1 = pd.concat(frames_y_mc_depl_minus1)

X_depl_mc_minus2 = pd.concat(frames_X_mc_depl_minus2)
y_depl_mc_minus2 = pd.concat(frames_y_mc_depl_minus2)

In [ ]:
y_train_mc_minus1 = y_train_mc_minus1.to_numpy().ravel()
y_train_mc_minus2 = y_train_mc_minus2.to_numpy().ravel()
y_valid_mc_minus1 = y_valid_mc_minus1.to_numpy().ravel()
y_valid_mc_minus2 = y_valid_mc_minus2.to_numpy().ravel()
# y_test_mc_minus1 = y_test_mc_minus1.to_numpy().ravel()
# y_test_mc_minus2 = y_test_mc_minus2.to_numpy().ravel()

In [ ]:
y_depl_mc_minus1 = y_depl_mc_minus1.to_numpy().ravel()
y_depl_mc_minus2 = y_depl_mc_minus2.to_numpy().ravel()

In [ ]:
y_depl_mc_minus2.shape

In [ ]:
X_train_nn.columns

In [ ]:
X_train_nn_minus1 = X_train_nn.drop(columns=['SnowCover', 'CriticalRecharge'])
X_valid_nn_minus1 = X_valid_nn.drop(columns=['SnowCover', 'CriticalRecharge'])
X_test_nn_minus1 = X_test_nn.drop(columns=['SnowCover', 'CriticalRecharge'])

X_train_nn_minus2 = X_train_nn.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain', 'Rain2D',
       'Rain3D', 'ExtremeRain', 'MonthlyRain', 'Tmin', 'Tmax', 'Tmean',
       'Trange', 'SnowTotal', 'Snow24h', 'Snow48h', 'Snow72h', 'MonthlySnow',
       'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])
X_valid_nn_minus2 = X_valid_nn.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain', 'Rain2D',
       'Rain3D', 'ExtremeRain', 'MonthlyRain', 'Tmin', 'Tmax', 'Tmean',
       'Trange', 'SnowTotal', 'Snow24h', 'Snow48h', 'Snow72h', 'MonthlySnow',
       'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])
X_test_nn_minus2 = X_test_nn.drop(columns=['SnowCover', 'CriticalRecharge', 'Rain', 'Rain2D',
       'Rain3D', 'ExtremeRain', 'MonthlyRain', 'Tmin', 'Tmax', 'Tmean',
       'Trange', 'SnowTotal', 'Snow24h', 'Snow48h', 'Snow72h', 'MonthlySnow',
       'WindSpeedMean', 'WindSpeedMax', 'Wind_South'])

In [ ]:
y_train_nn_minus1 = y_train_nn
y_train_nn_minus2 = y_train_nn
y_valid_nn_minus1 = y_valid_nn
y_valid_nn_minus2 = y_valid_nn
y_test_nn_minus1 = y_test_nn
y_test_nn_minus2 = y_test_nn

In [ ]:
frames_X_nn_depl_minus1 = [X_train_nn_minus1, X_valid_nn_minus1]
frames_y_nn_depl_minus1 = [y_train_nn_minus1, y_valid_nn_minus1]

frames_X_nn_depl_minus2 = [X_train_nn_minus2, X_valid_nn_minus2]
frames_y_nn_depl_minus2 = [y_train_nn_minus2, y_valid_nn_minus2]

In [ ]:
X_depl_nn_minus1 = pd.concat(frames_X_nn_depl_minus1)
y_depl_nn_minus1 = pd.concat(frames_y_nn_depl_minus1)

X_depl_nn_minus2 = pd.concat(frames_X_nn_depl_minus2)
y_depl_nn_minus2 = pd.concat(frames_y_nn_depl_minus2)

In [ ]:
y_train_nn_minus1 = y_train_nn.to_numpy().ravel()
y_train_nn_minus2 = y_train_nn.to_numpy().ravel()
y_valid_nn_minus1 = y_valid_nn.to_numpy().ravel()
y_valid_nn_minus2 = y_valid_nn.to_numpy().ravel()
# y_test_nn_minus1 = y_test_nn.to_numpy().ravel()
# y_test_nn_minus2 = y_test_nn.to_numpy().ravel()

In [ ]:
y_depl_nn_minus1 = y_depl_nn_minus1.to_numpy().ravel()
y_depl_nn_minus2 = y_depl_nn_minus2.to_numpy().ravel()

### 7) Support Vector Machine (SVM)
(They do well with small datasets (sometimes might need downsampling))

In [ ]:
from sklearn.svm import SVC

Build a preliminary SVM

In [ ]:
clf_svm = SVC(random_state=42)
clf_svm.fit(X_train_mc, y_train_mc)

Performance based on the VALIDATION SET

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_svm, X_valid_mc, y_valid_mc, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/SVM - CM before optimization', dpi=300, bbox_inches='tight')
plt.show();      # SVM are pretty good out of the box, confirmed!

Optimize parameters with Cross Validation and GridSearchCV(). \
\
Optimizing an SVM is all about finding the best value for gamma, and, potentially; the regularization parameter C.

Since we have two parameters to optimize we will use GridSearchCV().

In [ ]:
param_grid = [
    {'C': [0.5, 1, 10, 100],   # C must be > 0 
     'gamma': ['scale', 1, 0.1, 0.01, 0.001, 0.0001],
     'kernel': ['rbf', 'linear']}
]


optimal_params = GridSearchCV(SVC(),
                              param_grid,
                              cv=10,
                              scoring='accuracy',
                              verbose=0, 
                              n_jobs=-1)

optimal_params.fit(X_train_mc_minus1, y_train_mc_minus1)
print(optimal_params.best_params_)

Build the final SVM with the optimized parameters

In [ ]:
clf_svm = SVC(random_state=42, C=10, gamma='scale', kernel='rbf')
clf_svm.fit(X_depl_mc_minus1, y_depl_mc_minus1)

Performance on the VALIDATION SET

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_svm, X_valid_mc, y_valid_mc, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/SVM - CM after optimization', dpi=300, bbox_inches='tight')
plt.show();  

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf_svm, X_test_mc_minus1, y_test_mc_minus1, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/SVM - CM test', dpi=300, bbox_inches='tight')
plt.show()  

In [ ]:
score = np.mean(cross_val_score(clf_svm, X_test_mc_minus1, y_test_mc_minus1, scoring='accuracy', cv=10, n_jobs=-1))  
score

In [ ]:
y_pred = clf_svm.predict(X_test_mc)

In [ ]:
print(metrics.classification_report(y_test_mc, y_pred))

In [ ]:
print(metrics.recall_score(y_test, y_pred, pos_label=1))

The last thing we are going to do is to draw a support vector machine decision boundary to see how to interprete it.

**PCA (Principal Component Analysis)** \
\
We use PCA to combine the 35 features into 2 orthogonal meta-features that we can use as axes for a graph.
To put it simply, it is a way to scrink a 35-dimensional graph into a 2-dimensional graph. 

In [ ]:
# pca = PCA()  # By default, PCA() centers the data but does not scale it
# X_train_pca = pca.fit_transform(X_train)

# per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
# labels = [str(x) for x in range(1, len(per_var)+1)]

# plt.bar(x=range(1, len(per_var)+1), height=per_var)
# plt.tick_params(axis='x',
#                 which='both',
#                 bottom=False,
#                 top=False,
#                 labelbottom=False)
# plt.ylabel('Percentage of explained variance')
# plt.xlabel('Principal Components')
# plt.title('Scree plot')
# plt.show()

The bar plot shows that the first two components PC1 and PC2 account for a relatively large amount of variation in the raw data, and this means that they will be good candidates for the x and y axes in the 2-dimensional graph.

In [ ]:
# train_pc1_coords = X_train_pca[:, 0]
# train_pc2_coords = X_train_pca[:, 1]

# pca_train_scaled = scale(np.column_stack((train_pc1_coords, train_pc2_coords)))

# param_grid = [{'C': [0.5, 1, 10, 100],   # C must be > 0 
#                'gamma': ['scale', 1, 0.1, 0.01, 0.001, 0.0001],
#                'kernel': ['rbf', 'linear']}
# ]


# optimal_params = GridSearchCV(SVC(),
#                               param_grid,
#                               cv=10,
#                               scoring='accuracy',
#                               verbose=0)

# optimal_params.fit(pca_train_scaled, y_train)
# print(optimal_params.best_params_)

In [ ]:
# clf_svm = SVC(random_state=42, C=100, gamma='scale', kernel='rbf')
# clf_svm.fit(pca_train_scaled, y_train)

# X_valid_pca = pca.transform(X_valid)

# valid_pc1_coords = X_valid_pca[:, 0]
# valid_pc2_coords = X_valid_pca[:, 1]

# x_min = valid_pc1_coords.min() - 1
# x_max = valid_pc1_coords.max() + 1

# y_min = valid_pc2_coords.min() - 1
# y_max = valid_pc2_coords.max() + 1

# xx, yy = np.meshgrid(np.arange(start=x_min, stop=x_max, step=0.1),
#                      np.arange(start=y_min, stop=y_max, step=0.1))

# Z = clf_svm.predict(np.column_stack((xx.ravel(), yy.ravel())))
# Z = Z.reshape(xx.shape)

# fig, ax = plt.subplots(figsize=(10, 10))
# ax.contourf(xx, yy, Z, alpha=0.1)

# cmap = colors.ListedColormap(['#e41a1c', '#4daf4a'])

# scatter = ax.scatter(valid_pc1_coords, valid_pc2_coords, c=y_valid,
#                      cmap=cmap,
#                      s=100,
#                      edgecolors='k',  # k is black
#                      alpha=0.7)

# legend = ax.legend(scatter.legend_elements()[0],
#                    scatter.legend_elements()[1],
#                    loc='upper right')

# ax.set_ylabel('PC2')
# ax.set_xlabel('PC1')
# ax.set_title('Decision surface using the PCA transformed/projected features')
# plt.show()   # Doesn't look nice...Attempt not successful

### 8) Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
param_grid = [
    {'C': [0.5, 1, 10, 100],   # C must be > 0 
     'max_iter': [100, 500, 1000],
     'solver': ["lbfgs", "liblinear", "newton-cg", "newton-cholesky"]}
]

optimal_params = GridSearchCV(LogisticRegression(),
                              param_grid,
                              cv=10,
                              scoring='accuracy',
                              verbose=0, 
                              n_jobs=-1)

optimal_params.fit(X_train_mc_minus1, y_train_mc_minus1)
print(optimal_params.best_params_)

In [ ]:
reg = LogisticRegression(solver='lbfgs', max_iter = 100, C=1, random_state=5)
reg.fit(X_depl_mc_minus1, y_depl_mc_minus1)

Training accuracy

In [ ]:
y_pred = reg.predict(X_train_mc)
metrics.accuracy_score(y_train_mc, y_pred)

Validation accuracy

In [ ]:
ConfusionMatrixDisplay.from_estimator(reg, X_valid_mc, y_valid_mc, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Data_analysis/ML/cm_initial.png', dpi=300)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/log reg - CM', dpi=300, bbox_inches='tight')
plt.show();  

In [ ]:
y_pred = reg.predict(X_valid_mc)
metrics.accuracy_score(y_valid_mc, y_pred)

In [ ]:
y_pred

In [ ]:
print(metrics.classification_report(y_valid_mc, y_pred))

Testing accuracy

In [ ]:
ConfusionMatrixDisplay.from_estimator(reg, X_test_mc_minus2, y_test_mc_minus2, values_format = 'd', display_labels=['Not susceptible', 'Susceptible'], cmap=plt.cm.Blues)
#plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/log reg - CM test', dpi=300, bbox_inches='tight')
plt.show()  

In [ ]:
y_pred = reg.predict(X_test_mc)
metrics.accuracy_score(y_test_mc, y_pred)

In [ ]:
print(metrics.classification_report(y_test, y_pred))

In [ ]:
score = np.mean(cross_val_score(reg, X_test_mc_minus1, y_test_mc_minus1, scoring='accuracy', cv=10, n_jobs=-1))  
score

### 9) Feedforward Neural Network

In [ ]:
# tf.version.VERSION

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
len(X_train_nn.columns)

In [ ]:
X_train_nn   

Create the neural network

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim = len(X_depl_nn.columns), activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
print(model.summary())

Compile the model

(It checks if there are any cycles in the sequential model)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model

- We feed X_train into the model and the model calculates errors using y_train.
- In one epoch the model scans through all the rows in the X_train.
- Updating the number of epochs usually increases the accuracy of the model, because the more often the model sees the entire dataset the more likely it is that it's going to get better.
- To observe the accuracy on the validation data during the training, add validation_data = (X_valid, y_valid).

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
#callback_a = ModelCheckpoint(filepath = 'D:/Allaus/Code/my_best_NN.hdf5', monitor='val_loss', save_best_only=True, save_weights_only=True)
callback_b = EarlyStopping(monitor='val_loss', min_delta=0.0002, verbose=1, start_from_epoch=70)

In [ ]:
history = model.fit(X_train_nn_minus2, y_train_nn_minus2, validation_data=(X_valid_nn_minus2, y_valid_nn_minus2), epochs=256, batch_size=10, callbacks=[callback_a, callback_b])

Check the learning curves

In [ ]:
print(history.params)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['training data', 'validation data'], loc='lower right')
plt.show()

In [ ]:
model.load_weights('D:/Allaus/Code/my_best_NN.hdf5')

Evaluate the model on the TRAINING DATA
Just to make sure that everything works fine, **the score otherwise is useless**.

In [ ]:
scores = model.evaluate(X_train_nn, y_train_nn)
print(model.metrics_names)
print(scores)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Evaluate the model on the VALIDATION DATA

In [ ]:
scores = model.evaluate(X_valid_nn, y_valid_nn)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
prediction = model.predict(X_valid_nn)

In [ ]:
print(prediction[0:10].round())

In [ ]:
print(y_valid[0:10])

In [ ]:
plt.plot(y_valid_nn, prediction, '.', alpha=0.3)
plt.xlabel('True labels')
plt.ylabel('Predicted confidence scores')
plt.show()

Confusion matrix

In [ ]:
ax = plt.subplot()

cm = confusion_matrix(y_valid_nn, predict_results)

sns.heatmap(cm, annot=True, fmt='d', ax = ax, cmap=plt.cm.Blues)

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
#ax.set_title('Confusion matrix')
ax.xaxis.set_ticklabels(['Not susceptible', 'Susceptible'])
ax.yaxis.set_ticklabels(['Not susceptible', 'Susceptible'])
plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/NN - CM', dpi=300, bbox_inches='tight')
plt.show();

In [ ]:
cm

Evaluate the model on the TEST DATA

In [ ]:
scores = model.evaluate(X_test_nn, y_test_nn)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
prediction = model.predict(X_test_nn)

In [ ]:
predict_results = (prediction > 0.5)
#predict_results

Confusion matrix

In [ ]:
ax = plt.subplot()

cm = confusion_matrix(y_test_nn, predict_results)

sns.heatmap(cm, annot=True, fmt='d', ax = ax, cmap=plt.cm.Blues)

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
#ax.set_title('Confusion matrix')
ax.xaxis.set_ticklabels(['Not susceptible', 'Susceptible'])
ax.yaxis.set_ticklabels(['Not susceptible', 'Susceptible'])
plt.savefig('D:/Allaus/Manuscript/pictures_and_figures/NN - CM test', dpi=300, bbox_inches='tight')
plt.show();

Cross validation

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((X_depl_nn_minus2, X_test_nn_minus2), axis=0)
targets = np.concatenate((y_depl_nn_minus2, y_test_nn_minus2), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

    # Define the model architecture
    model = Sequential()
    model.add(Dense(64, input_dim = len(X_depl_nn_minus2.columns), activation = 'relu'))
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    print(model.summary())

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    from keras.callbacks import EarlyStopping, ModelCheckpoint
#     callback_a = ModelCheckpoint(filepath = 'D:/Allaus/Code/my_best_NN.hdf5', monitor='val_loss', save_best_only=True, save_weights_only=True)
    callback_b = EarlyStopping(monitor='val_loss', min_delta=0.0002, verbose=1, start_from_epoch=70)


    # Fit data to model
    history = model.fit(inputs[train], targets[train], validation_data=(inputs[test], targets[test]), epochs=256, batch_size=10, callbacks=callback_b)


    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
acc_per_fold

Other metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_valid, prediction.round())
precision = precision_score(y_valid, prediction .round(), pos_label=1)
recall = recall_score(y_valid, prediction.round(), pos_label=1)
f1score = f1_score(y_valid, prediction.round())
print("Accuracy: %.2f%%" % (accuracy*100.0))
print("Precision: %.2f%%" % (precision*100.0))
print("Recall: %.2f%%" % (recall*100.0))
print("F1-score: %.2f%%" % (f1score*100.0))

**How can the performance be improved?**

- Increase number of epochs.
- Add more layers into the neural network.
- Balance the data (in our case it is not really an issue).
- Increase/decrease the number of rows in the training/validation set.

## Compare models using heatmaps

In [ ]:
# Model evaluation

def evaluate(model, X_test, y_test, X_test_mc, y_test_mc, X_test_nn, y_test_nn, name):
    
    ypred=model.predict(xtest)  

    accuracy=np.round(balanced_accuracy_score(ytest,ypred),4)
    
    precision=np.round(precision_score(ytest,ypred,average = 'weighted'),4)

    recall=np.round(recall_score(ytest,ypred,average = 'weighted'),4)
    
    f1score=np.round(f1_score(ytest,ypred,average = 'weighted'),4)
    
#     cohenkappa_score=np.round(cohen_kappa_score(ytest,ypred),4)
 
#     matthews_corrcoef_=np.round(matthews_corrcoef(ytest,ypred),4)
    
    roc_auc = np.round(roc_auc_score(ytest, ypred), 4)
    
    return accuracy,precision,recall,f1score,roc_auc

In [ ]:
# Model fitting

def fit_data(X_depl, y_depl, X_test, y_test, X_depl_mc, y_depl_mc, X_test_mc, y_test_mc, X_depl_nn, y_depl_nn, X_test_nn, y_test_nn):
    
    # 1) Decision Tree
    clf_dt_pruned = DecisionTreeClassifier(random_state=42, ccp_alpha = 0.014) 
    clf_dt_pruned = clf_dt_pruned.fit(X_depl, y_depl)
    
    # 2) Random Forest Classifier
    clf_rf = RandomForestClassifier(criterion='entropy', max_depth=7, min_samples_split=4, n_estimators=139,
                       n_jobs=-1, random_state=42)
    clf_rf.fit(X_depl, y_depl)
        
    # 4) AdaBoost Classifier
    ada = AdaBoostClassifier(learning_rate=0.5, n_estimators=425, random_state=42)
    ada.fit(X_depl, y_depl)

    # 5) Gradient Boost
    GBC = GradientBoostingClassifier(max_depth=5, learning_rate=0.1, subsample=0.75, random_state=1, n_estimators=500)
    GBC.fit(X_depl, y_depl)
    
    # 6) XGBoost Classifier
    clf_xgb = xgb.XGBClassifier(objective='binary:logistic', 
                                seed=42, 
                                #early_stopping_rounds=10, 
                                #eval_metric='aucpr', 
                                gamma=0, 
                                learning_rate=0.3, 
                                max_depth=3, 
                                reg_lambda=1, 
                                scale_pos_weight=1,
                                #subsample=0.9, 
                                #colsample_bytree=0.5
                               )

    clf_xgb.fit(X_depl, 
                y_depl,
                verbose=True)#,
                #eval_set=[(X_test, y_test)])
        
    # SVM
    clf_svm = SVC(random_state=42, C=100, gamma=0.01, kernel='rbf')
    clf_svm.fit(X_depl_mc, y_depl_mc)
    
    # Logistic Regression
    reg = LogisticRegression(solver='lbfgs', max_iter = 100, C=1, random_state=5)
    reg.fit(X_depl_mc, y_depl_mc)
    
    # NN
    model.fit(X_depl_nn, y_depl_nn, epochs=256, batch_size=10)


    # this list will be used to store the scores for all classifiers
    performance_list=[]

    # performance metrics to be used for evaluating the classifiers
    performance_metrics=['Accuracy','Precision','Recall','F1 score', 'AUC score']
    
    indices=[]

    # create a dictionary object to store the models
    model_dict={'Decision Tree': clf_dt_pruned,'Random Forest': clf_rf, 'AdaBoost': ada, 
                'Gradient Boosting': GBC, 'XGBoost': clf_xgb, 'SVM': clf_svm, 'Logistic Regression': reg, 
                'Neural Network': model}

    # evaluate the each model stored in the dictionary object
    for name, model in model_dict.items():
        performance = evaluate(model, X_valid, y_valid, name)
        performance_list.append(performance)
        indices.append(name)
        
    performance_frame=pd.DataFrame(performance_list,columns=performance_metrics,index=indices)
    return performance_frame

In [ ]:
X_train.shape, X_valid.shape, X_test.shape

In [ ]:
# Fit the models to the training data and evaluate the models
# This is done by calling the functions created in the previous step
# The result is assigned to a variable

# Here I use directly the training set for evaluating the performance

result=fit_data(X_train, y_train, X_test, y_test)

In [ ]:
# get classifiers with f1score greater or equal to 0.9

result[result['Recall']>=0.9]

In [ ]:
# use seaborn to generate heatmap using the result from the previous step

plt.rcParams['figure.constrained_layout.use'] = True

sns.heatmap(result,annot=True,cmap='Blues')
plt.xlabel('Metric')
plt.ylabel('Classifiers')
plt.xticks(rotation=45)
plt.show()

## Compare models using ROC curves

In [ ]:
# Decision Tree
clf_dt_pruned = DecisionTreeClassifier(random_state=42, ccp_alpha = 0.014) 
clf_dt_pruned = clf_dt_pruned.fit(X_depl, y_depl)

In [ ]:
# Random Forest Classifier
clf_rf = RandomForestClassifier(criterion='entropy', max_depth=7, min_samples_split=4, n_estimators=139,
                       n_jobs=-1, random_state=42)
clf_rf.fit(X_depl, y_depl)

In [ ]:
# AdaBoost Classifier
ada = AdaBoostClassifier(learning_rate=0.5, n_estimators=425, random_state=42)
ada.fit(X_depl, y_depl)

In [ ]:
# Gradient Boost
GBC = GradientBoostingClassifier(max_depth=5, learning_rate=0.1, subsample=0.75, random_state=1, n_estimators=500)
GBC.fit(X_depl, y_depl)

In [ ]:
# XGBoost Classifier
clf_xgb = xgb.XGBClassifier(objective='binary:logistic', 
                            seed=42, 
                            #early_stopping_rounds=10, 
                            #eval_metric='aucpr', 
                            gamma=0, 
                            learning_rate=0.3, 
                            max_depth=3, 
                            reg_lambda=1, 
                            scale_pos_weight=1,
                            #subsample=0.9, 
                            #colsample_bytree=0.5
                           )

clf_xgb.fit(X_depl, 
            y_depl,
            verbose=True)#,
            #eval_set=[(X_test, y_test)])

In [ ]:
# SVM
clf_svm = SVC(random_state=42, C=100, gamma=0.01, kernel='rbf')
clf_svm.fit(X_depl_mc, y_depl_mc)

In [ ]:
# Logistic Regression
reg = LogisticRegression(solver='lbfgs', max_iter = 100, C=1, random_state=5)
reg.fit(X_depl_mc, y_depl_mc)

In [ ]:
# NN
history = model.fit(X_depl_nn, y_depl_nn, validation_data=(X_test_nn, y_test_nn), epochs=256, batch_size=10, callbacks=callback_b)


Plot ROC Curve for each classifier


In [ ]:
# def plot_roc(xtest,ytest,models):
    
#     #models object should be a dictionary comprising of name of model and the model object
#     for name,model in models.items():

#         if hasattr(model,'decision_function'):
#             probs=model.decision_function(xtest) 
#         elif hasattr(model,'predict_proba'):
#             probs=model.predict_proba(xtest) [:,1]
#         fpr,tpr,threshold=roc_curve(ytest,probs)
#         roc_auc=auc(fpr,tpr)
#         print('ROC AUC=%0.2f'%roc_auc)
#         plt.plot(fpr,tpr,label='%s (AUC=%0.2f)'%(name,roc_auc))
        
#     plt.legend(loc='lower right')
#     plt.plot([0,1],[0,1],'b--')
#     plt.xlim([0,1])
#     plt.ylim([0,1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.show()

In [ ]:
def plot_roc(X_test, y_test, X_test_mc, y_test_mc, X_test_nn, y_test_nn, models):
    
    #models object should be a dictionary comprising of name of model and the model object
    for name,model in models.items():

        if (name == 'Logistic Regression') or (name == 'SVM'):
            probs = model.predict(X_test_mc)
        elif name == 'Neural Network':
            probs = model.predict(X_test_nn)
        else:
            probs = model.predict(X_test)
            
        if (name == 'Logistic Regression') or (name == 'SVM'):
            fpr,tpr,threshold=roc_curve(y_test_mc,probs)
        elif name == 'Neural Network':
            fpr,tpr,threshold=roc_curve(y_test_nn,probs)
        else:
            fpr,tpr,threshold=roc_curve(y_test,probs)

        roc_auc=auc(fpr,tpr)
        print('ROC AUC=%0.2f'%roc_auc)
        plt.plot(fpr,tpr,label='%s (AUC=%0.2f)'%(name,roc_auc))
        
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'b--')
    plt.xlim([0,1])
    plt.ylim([0,1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()

Based on VALIDATION

In [ ]:
models={'Decision Tree': clf_dt_pruned,'Random Forest': clf_rf, 'AdaBoost': ada, 
        'Gradient Boost': GBC, 'XGBoost': clf_xgb, 'SVM': clf_svm, 'Logistic Regression': reg, 
        'Neural Network': model}
plot_roc(X_test, y_test, X_test_mc, y_test_mc, X_test_nn, y_test_nn, models) 

# the steepness of the roc curve also helps us get the performance
# apart from the auc that is plotted in the graph

Based on TEST

In [ ]:
models={'Decision Tree': clf_dt_pruned,'Random Forest': clf_rf, 'AdaBoost': ada, 
        'Gradient Boost': GBC, 'XGBoost': clf_xgb}
plot_roc(X_test, y_test, models) 